In [3]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [9]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt


In [10]:
from transformers import TFXLNetForSequenceClassification, XLNetTokenizerFast


In [13]:
'''from google.colab import drive



# Mount Google Drive

drive.mount('/content/drive')
'''

"from google.colab import drive\n\n\n\n# Mount Google Drive\n\ndrive.mount('/content/drive')\n"

In [15]:
import pandas as pd

combined_df = pd.read_csv("/kaggle/input/combined-df/combined_df.csv")
combined_df

,label,text
0,1,CPPRI Recruitment 2010 at http://ping.fm/yp8zH
1,1,National Games Secretariat Recruitment 2010 :...
2,1,CIPET Recruitment Jobs at http://ping.fm/KnFCa
3,1,DIAT Recruitment 2010 at http://ping.fm/huS9m
4,1,BHEL Recruitment 2010 : http://ping.fm/PLWWA
...,...,...
1193663,0,http://t.co/pzOicLAz
1193664,0,you ain't nothin but a hoochie mamma.... hood ...
1193665,0,watching The Sweetest Thing!!!!
1193666,0,Looking like a library kind of day


In [16]:
combined_df["label"].value_counts()

label
0    1048574
1     145094
Name: count, dtype: int64

In [17]:
import pandas as pd

# Step 1: Find the number of bot samples
num_bots = combined_df[combined_df['label'] == 1].shape[0]

# Step 2: Randomly select the same number of genuine samples
genuine_samples = combined_df[combined_df['label'] == 0].sample(n=num_bots, random_state=42)

# Step 3: Combine the undersampled genuine samples with all bot samples
undersampled_df = pd.concat([genuine_samples, combined_df[combined_df['label'] == 1]], ignore_index=True)

# Shuffle the combined dataset
undersampled_df = undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print(undersampled_df["label"].value_counts())


label
1    145094
0    145094
Name: count, dtype: int64


In [18]:
import re

# Preprocessing function

def preprocess_text(text):
    text = text.lower()
    # Replace special Twitter elements with predefined tokens
    text = re.sub(r'@\w+', 'USER', text)  # Replace mentions with USER
    text = re.sub(r'#\w+', 'HASHTAG', text)  # Replace hashtags with HASHTAG
    text = re.sub(r'http\S+', 'URL', text)  # Replace URLs with URL
    return text

In [19]:
undersampled_df['text'] = undersampled_df['text'].astype(str).apply(preprocess_text)


In [20]:
# Tokenize and prepare dataset using the single tokenizer
def tokenize_data(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='tf')


In [21]:
# Split the data
from sklearn.model_selection import train_test_split
train_tweets, test_tweets, train_labels, test_labels = train_test_split(
    undersampled_df['text'].tolist(),
    undersampled_df['label'].tolist(),
    test_size=0.2,
    random_state=42

)

In [22]:
from transformers import TFXLNetForSequenceClassification, XLNetTokenizerFast

# Initialize the tokenizer for XLNet
tokenizer = XLNetTokenizerFast.from_pretrained("xlnet-base-cased")


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
# Convert to TensorFlow dataset
train_encodings = tokenize_data(train_tweets)
test_encodings = tokenize_data(test_tweets)


In [26]:
from transformers import TFXLNetForSequenceClassification, XLNetTokenizerFast

# Initialize the model for sequence classification
model = TFXLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=1)


/opt/conda/lib/python3.10/site-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Be

In [27]:
pip show tensorflow 

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: explainable-ai-sdk, tensorflow-cloud, tensorflow-serving-api, tensorflow-text, tensorflow_decision_forests, tf_keras, witwidget
Note: you may need to restart the kernel to use updated packages.


In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).shuffle(100).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(32)


In [30]:

# Compile the model
learning_rate = 5e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy(name="accuracy")]
)


In [31]:
import tensorflow as tf
from transformers import TFXLNetForSequenceClassification
from tensorflow.keras.callbacks import EarlyStopping
# Create the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



In [32]:
'''from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

'''

"from tensorflow.keras.callbacks import EarlyStopping\nearly_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)\n\n"

In [33]:
import tensorflow as tf
import keras
from transformers import __version__ as transformers_version

print(tf.__version__)
print(keras.__version__)
print(transformers_version)


2.16.1
3.3.3
4.45.1


In [35]:
# Train the model
epochs = 2
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)

Epoch 1/2
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1730631589.378970     265 service.cc:145] XLA service 0x7ea6046bbbc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730631589.379016     265 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730631589.379020     265 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1730631589.653640     265 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7255/7255 [==============================] - 7863s 1s/step - loss: 0.3691 - accuracy: 0.7735 - val_loss: 0.6948 - val_accuracy: 0.4997
Epoch 2/2
7255/7255 [==============================] - 7764s 1s/step - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.7087 - val_accuracy: 0.4997


In [36]:
# Evaluate the model on the test set

eval_results = model.evaluate(test_dataset)
print(f"Test loss: {eval_results[0]}, Test accuracy: {eval_results[1]}")


1814/1814 [==============================] - 612s 337ms/step - loss: 0.7087 - accuracy: 0.4997
Test loss: 0.7087351679801941, Test accuracy: 0.49968984723091125


In [37]:
# Evaluate the model on the training set to calculate training accuracy
train_eval_results = model.evaluate(train_dataset)
print(f"Train loss: {train_eval_results[0]}, Train accuracy: {train_eval_results[1]}")


7255/7255 [==============================] - 2449s 337ms/step - loss: 0.7086 - accuracy: 0.5001
Train loss: 0.708602786064148, Train accuracy: 0.5000775456428528
